In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

In [5]:
data = load_wine()
x, y = data.data, data.target

In [6]:
df = pd.DataFrame(x, columns=data.feature_names)
df['target']= y
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
#-------------- Menggunakan SelectKbeast ---------------------
filter_selector = SelectKBest(score_func=chi2, k=4)
X_train_filtered = filter_selector.fit_transform(X_train, y_train)
X_test_filtered = filter_selector.transform(X_test)
print("Fitur yang dipilih dengan filter Methods: ", filter_selector.get_support(indices=True))

Fitur yang dipilih dengan filter Methods:  [ 4  6  9 12]


In [11]:
#--------------Wrapper Methods------------------
model = LogisticRegression(solver='lbfgs', max_iter=5000)
rfe_selector = RFE(estimator=model, n_features_to_select=4)
X_train_rfe = rfe_selector.fit_transform(X_train, y_train)
X_test_rfe = rfe_selector.transform(X_test)
print("Fitur yang dipilih dengan Wrapper Methods:", rfe_selector.get_support(indices=True))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitur yang dipilih dengan Wrapper Methods: [ 0  6  9 11]


In [16]:
#----------------Embedded Methods------------
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
feature_importances = rf_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
threshold = 0.05
important_feature_indices = [i for i in range(len(feature_importances)) if feature_importances[i] >= threshold]
X_important = X_train[:, important_feature_indices]
X_test_important = X_test[:, important_feature_indices]

print("Fitur yang dipilih dengan Embedded Methods (diatas amabang batas):")
for i in important_feature_indices:
    print(f"{data.feature_names[i]}: importance={feature_importances[i]}")
print("\nDimensi data pelatihan dengan fitur penting", X_important.shape)
print("Dimensi data pengujian dengan fitur penting", X_test_important.shape)

Fitur yang dipilih dengan Embedded Methods (diatas amabang batas):
alcohol: importance=0.11239773542143086
flavanoids: importance=0.20229341635663622
color_intensity: importance=0.1712021830864957
hue: importance=0.07089132259413944
od280/od315_of_diluted_wines: importance=0.1115643167260497
proline: importance=0.13904586955351153

Dimensi data pelatihan dengan fitur penting (142, 6)
Dimensi data pengujian dengan fitur penting (36, 6)


Evaluasi antara 3 jenis extraciton selection yang dipilih

---



In [17]:
def evaluate_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    return accuracy

In [22]:
logistic_model_filter = LogisticRegression(max_iter=200)
accuracy_logistic_filter = evaluate_model(X_train_filtered, X_test_filtered, y_train, y_test, logistic_model_filter)
print("Akurasi Logistic Regression dengan Filter Methods:", accuracy_logistic_filter)
logistic_model_rfe = LogisticRegression(max_iter=200)
accuracy_logistic_wrapper = evaluate_model(X_train_rfe, X_test_rfe, y_train, y_test, logistic_model_rfe)
print("Akurasi Logistic Regression dengan Wrapper Methods:", accuracy_logistic_wrapper)

accuracy_rf = evaluate_model(X_important, X_test_important, y_train, y_test, rf_model)
print("Akurasi Random Forest:", accuracy_rf)

Akurasi Logistic Regression dengan Filter Methods: 0.9722222222222222
Akurasi Logistic Regression dengan Wrapper Methods: 0.9722222222222222
Akurasi Random Forest: 1.0
